In [1]:
import numpy as np
import optimizer
import sinkhorn_ops
import matplotlib.pyplot as plt
from scipy.misc import imresize
from scipy.optimize import linear_sum_assignment
%matplotlib inline
from time import time
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import numpy as np

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
data_train = input_data.read_data_sets('/tmp/', one_hot=True).train 
data_test = input_data.read_data_sets('/tmp/', one_hot=True).test 


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /tmp/t10k-images-idx3-ubyte.gz
Extracting /tmp/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Extracting /tmp/train-images-idx3-ubyte.gz
Extracting /tmp/train-labels-idx1-ubyte.gz
Extracting /tmp/t10k-images-idx3-ubyte.gz
Extracting /tmp/t10k-labels-idx1-ubyte.gz


In [2]:
from scipy.special import logsumexp
def sinkhorn(log_alpha, n_iters=20):

    for n in range(n_iters):
        log_alpha -= logsumexp(log_alpha, axis=1, keepdims=True)
        log_alpha -= logsumexp(log_alpha, axis=0, keepdims=True)
        if n%10000==1:
            print(n)
    log_alpha -= logsumexp(log_alpha, axis=1, keepdims=True)
    return np.exp(log_alpha)

def batch_split(batch, n_squares_side, n_channels=1):
    if(n_channels ==1):
        #side = int(np.sqrt(batch.shape[1]))
        side =batch.shape[1]
    else:
        side = batch.shape[1]
    batch_size = batch.shape[0]
    n_squares = n_squares_side ** 2
    
    batch = np.reshape(batch, [-1, side, side, n_channels])
    batch = np.reshape(batch, [batch_size, n_squares_side, side/n_squares_side, side, n_channels])
    batch = np.transpose(batch, [0, 2, 1, 3, 4])
    batch = np.reshape(batch, [batch_size, side/n_squares_side, n_squares, side/n_squares_side, n_channels])
    batch = np.transpose(batch, [0, 2, 1, 3, 4])
    return batch

def stack_batch_split(batch):
    return np.reshape(batch, [batch.shape[0]*batch.shape[1], batch.shape[2], batch.shape[3], batch.shape[4]])


def unflatten_batch(batch, n_channels=1):
    side_square = int(np.sqrt(batch.shape[2]/n_channels))
    return np.reshape(batch, [batch.shape[0], batch.shape[1], side_square, side_square, n_channels])

def join_batch_split(batch):
    batch_size = batch.shape[0]
    n_squares = batch.shape[1]
    side_quare = batch.shape[2]
    n_channels = batch.shape[4]
    n_squares_side = int(np.sqrt(n_squares))
    batch = np.transpose(batch, [0, 1, 3, 2, 4])
    batch = np.reshape(batch, [batch_size, n_squares_side, side_square*n_squares_side, side_square, n_channels])
    batch = np.transpose(batch, [0,1, 3,2,4])
    batch = np.reshape(batch, [batch_size, 1, side_square*n_squares_side, side_square*n_squares_side, n_channels])
    batch = np.reshape(batch, [batch_size, side_square*n_squares_side, side_square*n_squares_side, n_channels])
    return batch

def resized_dims(n_squares_side):
    if(n_squares_side==2):
        side = 28
        side_square = 14
    if(n_squares_side==3):
        side = 27
        side_square = 9
    if(n_squares_side==4):
        side = 28
        side_square = 7
    if(n_squares_side==5):
        side = 30
        side_square = 6
    if(n_squares_side==6):
        side = 30
        side_square = 5
    if(n_squares_side==7):
        side = 28
        side_square = 4
    if(n_squares_side==8):
        side = 32
        side_square = 4
    if(n_squares_side==9):
        side = 27
        side_square = 3
    if(n_squares_side==16):
        side = 80
        side_square = 5
    if(n_squares_side==14):
        side = 196
        side_square = 14
    if(n_squares_side==18):
        side = 8*18
        side_square = 8
    if(n_squares_side==20):
        side = 7*20
        side_square = 7
    if(n_squares_side==25):
        side = 25*6
        side_square = 6
    if(n_squares_side==30):
        side = 30*5
        side_square = 5
    if (n_squares_side == 37):
        side = 37 * 4
        side_square = 4
    if (n_squares_side == 45):
        side = 45 * 3
        side_square = 3
    return side, side_square

def resize_batch_color(batch, side_new, n_channels):
    batch_new = np.zeros((batch.shape[0], side_new, side_new, n_channels))
    side = int(np.sqrt(batch.shape[1]))
    for i in range(batch.shape[0]):
        for c in range(n_channels):
            a = imresize(batch[i,:,:,c], [side_new, side_new])
            
            a = a/255.0
            batch_new[i,:,:,c] =a
    return batch_new


def soft_to_hard(soft_perm):
    
    a,b = linear_sum_assignment(-soft_perm)
    ma = np.zeros((np.shape(soft_perm)))
    for i in range(soft_perm.shape[0]):
        ma[i, b[i]] = 1
    return ma


def load_image( infilename ) :
    img = Image.open( infilename )
    img.load()
    data = np.asarray( img, dtype="int32" )
    return data


In [3]:
# Define model params
batch_size = 20
n_iter_sinkhorn2 = 10
n_iter_sinkhorn = 200

temp = 0.1
temp2 =0.1

#mnist data
n_squares_side = 16
lr = 0.0001
n_channels = 1
rfield_size = 3
stride = 3
n_units = 5
keep_prob = 1.0
side_real = 28
opt = 'adam'
samples_per_num = 1
n_squares = n_squares_side **2
side, side_square = resized_dims(n_squares_side)
n_dim = int(side_square*side_square*n_channels)
print(n_dim)
n_dim_z = 512
n_dim_z2 = 128
n_dim_zp = 64
noise_factor = 0
np.random.seed(2)



25


In [4]:
#Now we define the main TF variables


real_split = tf.placeholder(tf.float32,[None, n_squares, side_square, side_square, n_channels])
real_split_tiled = tf.tile(real_split, [samples_per_num, 1, 1, 1, 1])
real_split_tiled = tf.reshape(real_split_tiled, [-1, n_squares, side_square ** 2* n_channels])

real_entire = tf.placeholder(tf.float32,[None, side_real, side_real, n_channels])
real_entire_tiled = tf.tile(real_entire,[samples_per_num,1,1,1])

global_step = tf.Variable(0, trainable=False)
stack_im = tf.placeholder(tf.float32,[None, side_square, side_square, 1])



Instructions for updating:
Colocations handled automatically by placer.


In [5]:

def create_z():
    #create the matrix of log_alpha, that will later will converted into a soft permutation
    #this relies on some NN processing (convolutional), see below
    fc = tf.contrib.layers.fully_connected
    flatten = tf.contrib.layers.flatten
    dropout = tf.contrib.layers.dropout
    def conv(input_image, kernel_shape, bias_shape):
        weights = tf.get_variable("weights", kernel_shape,
                                 initializer = tf.random_normal_initializer())
        biases = tf.get_variable("biases", bias_shape, 
                                initializer=tf.constant_initializer(-1))
        convolutional = tf.nn.conv2d(input_image, weights, 
                                     strides = [1, 1, 1, 1],
                                     padding="SAME")
        out_relu = tf.nn.relu(convolutional + biases)
        out_maxpool = tf.nn.max_pool(out_relu, 
                                    ksize=[1, stride, stride, 1],
                                   strides=[1, stride, stride, 1],
                                   padding="SAME")
        return out_maxpool
    def conv_and_fc(input_image, scope, n_dim_z):
        with tf.variable_scope(scope):
            conv_output = conv(input_image, [rfield_size, rfield_size, n_channels, n_units], [n_units])
        fully_connected_output_mean = dropout(tf.cast(fc(flatten(conv_output), n_dim_z, activation_fn = None), tf.float32),
                                        keep_prob)
        fully_connected_output_log_var = dropout(tf.cast(fc(flatten(conv_output), n_dim_z, activation_fn = None), tf.float32),
                                        keep_prob)
      
        return fully_connected_output_mean, fully_connected_output_log_var
    
    #with tf.variable_scope("model_params"):
    z_mean, z_log_var = conv_and_fc(real_entire_tiled,'n1', n_dim_z)
    z_mean_pieces, z_log_var_pieces = conv_and_fc(real_entire_tiled,'n2',n_dim_zp)
    #z_mean_pieces, z_log_var_pieces = conv_and_fc(stack_im,'n2',n_dim_zp)
    z_mean_pieces = tf.reshape(tf.tile(z_mean_pieces, [1, n_squares]), [batch_size*n_squares,n_dim_zp])
    #z_mean_pieces = tf.reshape(z_mean_pieces, [1, n_squares]), [batch_size*n_squares,n_dim_zp])
    
    z_log_var_pieces = tf.reshape(tf.tile(z_log_var_pieces, [1, n_squares]), [batch_size*n_squares,n_dim_zp])
    z_pieces = z_mean_pieces + tf.random.normal([batch_size*n_squares, n_dim_zp]) *tf.exp(z_log_var_pieces/2)
    aa = fc(z_pieces,1, activation_fn = None)
    
    z_pieces_fc = tf.contrib.distributions.Bernoulli(logits=aa/temp).mean()
    scrambled_split_tiled = tf.tile(tf.reshape(z_pieces_fc, [batch_size, n_squares, 1]), [1,1, n_channels*side_square**2])
    
    z  = z_mean + tf.random.normal([batch_size, n_dim_z]) *tf.exp(z_log_var/2)
    z1 = tf.reshape(fc(z, n_squares*n_dim_z2, activation_fn = None), [-1, n_squares, n_dim_z2])
    #z2 = tf.reshape(fc(flatten(z_pieces), n_dim_z2, activation_fn = None), [-1,  n_dim_z2,  n_squares])
    z2 = tf.reshape(fc(flatten(z_pieces_fc), n_dim_z2, activation_fn = None), [-1,  n_dim_z2,  n_squares])
    
    cost = tf.nn.sigmoid(tf.matmul(z1,z2))
   
    return z,z_pieces, z_mean, z_log_var, z_mean_pieces, z_log_var_pieces, cost, scrambled_split_tiled, aa,z_pieces_fc

In [6]:


#ordered_inf = tf.matmul(inv_soft_perms_flat, scrambled_split_tiled)

z,z_pieces, z_mean, z_log_var, z_mean_pieces, z_log_var_pieces, cost, scrambled_split_tiled,aa,z_pieces_fc = create_z()
(soft_perms_inf2, _) = sinkhorn_ops.gumbel_sinkhorn(-cost, temp2, samples_per_num, noise_factor, n_iter_sinkhorn2, squeeze=False)
inv_soft_perms2 = tf.transpose(soft_perms_inf2, [0, 1, 3, 2])
inv_soft_perms2_flat = tf.reshape(tf.transpose(inv_soft_perms2, [1, 0, 2, 3]), [-1, n_squares, n_squares])

ordered_inf2 = tf.matmul(inv_soft_perms2_flat, scrambled_split_tiled)
 
rec_loss1 =tf.reduce_mean(tf.square(ordered_inf2 - real_split_tiled))
rec_loss =-tf.reduce_mean(tf.reduce_mean(real_split_tiled*tf.log(1e-5+ordered_inf2) +(1-real_split_tiled)*tf.log(1e-5+1-ordered_inf2),axis=0))*side_real**2

KL1 = 0.5 *tf.reduce_mean(tf.reduce_sum(1+z_log_var -tf.square(z_mean) -tf.exp(z_log_var), axis=1))
KL2 = 0.5 *tf.reduce_mean(tf.reduce_sum(1+z_log_var_pieces -tf.square(z_mean_pieces) -tf.exp(z_log_var_pieces), axis=1))
KL = KL1+KL2*n_squares
l2s_diff = rec_loss - KL

#l2s_diff = tf.reduce_mean(tf.square(ordered_inf2 - real_split_tiled))

opt = optimizer.set_optimizer(opt, lr, opt_eps=1e-8)
train_op = tf.contrib.training.create_train_op(l2s_diff, opt, global_step=global_step)


Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
Use tf.cast instead.


In [ ]:

init_op=tf.initialize_all_variables()
sess=tf.InteractiveSession()
sess.run(init_op)
new = tf.trainable_variables()
print(new)
n_iter = 20000
for i in range(n_iter):
    if i<n_iter-1:
        ims,_=data_train.next_batch(batch_size)
    else:
        print('hola')
        ims,_=data_test.next_batch(batch_size)        
    ims = np.expand_dims(np.reshape(ims, [-1, 28,28]),axis=3)
    ims[ims>0.5]=1
    ims[ims<0.5]=0
    np_x = resize_batch_color(ims, side, n_channels)

    real_images_split = batch_split(np_x, n_squares_side, n_channels)
    stack_real_images_split = stack_batch_split(real_images_split)
    [o2,loss, rec_loss_np,KL_np,rec_loss1_np,_, sp2,cos,z_mean_pieces_np, z_pieces_fc_np,z_log_var_pieces_np,aa_np] = sess.run([ordered_inf2, l2s_diff,rec_loss,KL,rec_loss1,train_op, soft_perms_inf2,cost,z_mean_pieces,z_pieces_fc,z_log_var_pieces,aa],{real_split:real_images_split,real_entire:ims, stack_im:stack_real_images_split})
    if(i%10==0):
        print([i, loss, rec_loss_np, KL_np, rec_loss1_np])
    if i==n_iter-1:
        ogen = []
        sgen =[]
        for k in range(5):
            [oo2,ssp2,scrambled_pieces_split_np] = sess.run([ordered_inf2,soft_perms_inf2, scrambled_split_tiled],{z:np.random.normal(0,1,(batch_size, n_dim_z)),
                                                                 z_pieces:np.random.normal(0,1,(n_squares*batch_size, n_dim_zp))})
            ogen.append(oo2)
            sgen.append(ssp2)

   


Instructions for updating:
Use `tf.global_variables_initializer` instead.
[<tf.Variable 'n1/weights:0' shape=(3, 3, 1, 5) dtype=float32_ref>, <tf.Variable 'n1/biases:0' shape=(5,) dtype=float32_ref>, <tf.Variable 'fully_connected/weights:0' shape=(500, 512) dtype=float32_ref>, <tf.Variable 'fully_connected/biases:0' shape=(512,) dtype=float32_ref>, <tf.Variable 'fully_connected_1/weights:0' shape=(500, 512) dtype=float32_ref>, <tf.Variable 'fully_connected_1/biases:0' shape=(512,) dtype=float32_ref>, <tf.Variable 'n2/weights:0' shape=(3, 3, 1, 5) dtype=float32_ref>, <tf.Variable 'n2/biases:0' shape=(5,) dtype=float32_ref>, <tf.Variable 'fully_connected_2/weights:0' shape=(500, 64) dtype=float32_ref>, <tf.Variable 'fully_connected_2/biases:0' shape=(64,) dtype=float32_ref>, <tf.Variable 'fully_connected_3/weights:0' shape=(500, 64) dtype=float32_ref>, <tf.Variable 'fully_connected_3/biases:0' shape=(64,) dtype=float32_ref>, <tf.Variable 'fully_connected_4/weights:0' shape=(64, 1) dtype=

/home/gomena/.local/lib/python2.7/site-packages/ipykernel_launcher.py:105: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.3.0.
Use Pillow instead: ``numpy.array(Image.fromarray(arr).resize())``.


[0, 1802.9858, 490.5307, -1312.4552, 0.19887356]
[10, 1838.5322, 429.976, -1408.5562, 0.16016233]
[20, 1719.613, 441.85577, -1277.7573, 0.16680539]
[30, 1629.8552, 467.79254, -1162.0627, 0.18329947]
[40, 1366.8062, 410.15076, -956.65533, 0.15026183]
[50, 1513.8372, 398.75235, -1115.0848, 0.14205898]
[60, 1316.1973, 396.59155, -919.60565, 0.14232484]
[70, 1434.813, 406.04517, -1028.7678, 0.14640944]
[80, 1342.4132, 380.26117, -962.15204, 0.13201733]
[90, 1238.5295, 367.17334, -871.35614, 0.1255748]
[100, 1188.7092, 391.79425, -796.915, 0.14012411]
[110, 1304.1938, 372.65274, -931.54114, 0.12578097]
[120, 1151.6938, 378.92184, -772.772, 0.13230152]
[130, 1259.3027, 393.07965, -866.22314, 0.1394859]
[140, 1056.0449, 387.61014, -668.4348, 0.13619463]
[150, 1151.8042, 375.93085, -775.8734, 0.12940992]
[160, 1139.2202, 359.56604, -779.6542, 0.12021875]
[170, 1088.6777, 370.51343, -718.16425, 0.12706695]
[180, 1096.457, 370.81055, -725.64655, 0.12692967]
[190, 1125.1515, 371.58725, -753.5642,

[1600, 481.5475, 346.35513, -135.19234, 0.112352535]
[1610, 455.82593, 331.07117, -124.754776, 0.10522632]
[1620, 446.12347, 314.58835, -131.53511, 0.09822117]
[1630, 473.71558, 346.35614, -127.35944, 0.113500096]
[1640, 473.75403, 335.90228, -137.85173, 0.10740586]
[1650, 448.0857, 317.39774, -130.68794, 0.09745173]
[1660, 456.9228, 322.42218, -134.50063, 0.09989116]
[1670, 433.73743, 317.29327, -116.44417, 0.099059545]
[1680, 459.6944, 335.86563, -123.82877, 0.10794741]
[1690, 452.8668, 322.70883, -130.15796, 0.10102197]
[1700, 456.83368, 330.13696, -126.69673, 0.105924346]
[1710, 459.60187, 337.79318, -121.80869, 0.11030717]
[1720, 448.9392, 323.68506, -125.254166, 0.10255847]
[1730, 438.376, 327.56937, -110.80665, 0.10469032]
[1740, 482.82852, 342.6268, -140.20172, 0.11149252]
[1750, 457.15723, 335.73044, -121.426796, 0.1078611]
[1760, 456.30865, 321.27673, -135.03192, 0.10142123]
[1770, 451.24365, 320.32635, -130.91728, 0.09990001]
[1780, 445.05487, 327.1245, -117.93036, 0.1028819

[3180, 328.89813, 273.21643, -55.6817, 0.08280254]
[3190, 306.11343, 248.20671, -57.906734, 0.07189937]
[3200, 321.81348, 268.4914, -53.322067, 0.083898485]
[3210, 313.245, 262.53568, -50.709328, 0.07991873]
[3220, 330.55927, 275.5037, -55.055588, 0.0843051]
[3230, 337.1504, 278.7845, -58.36589, 0.08697307]
[3240, 322.7419, 264.35382, -58.388084, 0.07816951]
[3250, 321.4303, 262.51443, -58.915863, 0.07788716]
[3260, 335.0201, 280.84805, -54.17205, 0.088324115]
[3270, 335.27985, 281.37036, -53.9095, 0.08966852]
[3280, 324.02606, 270.14703, -53.879044, 0.0837609]
[3290, 328.4105, 269.8369, -58.57357, 0.08123612]
[3300, 327.09503, 273.76382, -53.331223, 0.084748074]
[3310, 313.75098, 260.70813, -53.042854, 0.07896238]
[3320, 331.60507, 283.01776, -48.58731, 0.08817571]
[3330, 316.76, 264.36078, -52.399246, 0.08171968]
[3340, 308.05038, 254.99701, -53.053368, 0.07576179]
[3350, 332.03723, 280.26205, -51.775192, 0.08762837]
[3360, 323.1436, 274.02527, -49.11832, 0.08489032]
[3370, 308.55066

[4750, 259.97144, 244.62361, -15.34783, 0.073379986]
[4760, 271.77444, 258.54437, -13.230068, 0.081131]
[4770, 240.72649, 226.68196, -14.044523, 0.066323444]
[4780, 242.2511, 228.77261, -13.478481, 0.06959889]
[4790, 257.35233, 244.82646, -12.525855, 0.07581641]
[4800, 253.18584, 239.06052, -14.125319, 0.07304417]
[4810, 245.5391, 231.72101, -13.818077, 0.07040196]
[4820, 261.37503, 248.11005, -13.264992, 0.07539113]
[4830, 239.82182, 227.83594, -11.985884, 0.06707737]
[4840, 254.85718, 242.9864, -11.870774, 0.07370972]
[4850, 259.26413, 246.88847, -12.375668, 0.075915396]
[4860, 256.36362, 245.60092, -10.762689, 0.07496164]
[4870, 264.62857, 251.86977, -12.758791, 0.07912181]
[4880, 258.7183, 245.4793, -13.239001, 0.0744461]
[4890, 245.27417, 232.811, -12.463165, 0.068758875]
[4900, 240.15134, 227.03758, -13.113761, 0.06764581]
[4910, 267.48712, 254.58461, -12.902527, 0.07775295]
[4920, 249.25641, 236.28468, -12.971728, 0.07089025]
[4930, 257.20526, 244.08417, -13.121109, 0.074324846]

[6320, 249.42918, 240.3851, -9.044088, 0.07586019]
[6330, 253.29405, 244.58821, -8.705838, 0.074088305]
[6340, 246.37468, 238.05287, -8.321807, 0.072874784]
[6350, 266.1912, 256.75122, -9.4399605, 0.0824778]
[6360, 249.56999, 240.47736, -9.092643, 0.07415687]
[6370, 261.95734, 251.5833, -10.374026, 0.07817076]
[6380, 245.9738, 236.99037, -8.983425, 0.075638354]
[6390, 237.64345, 227.91846, -9.724995, 0.07057472]
[6400, 249.48587, 240.44904, -9.036841, 0.07588474]
[6410, 257.0192, 247.24924, -9.769948, 0.07491334]
[6420, 249.33873, 240.09755, -9.241181, 0.07287506]
[6430, 254.77982, 245.54146, -9.238357, 0.07642841]
[6440, 244.65762, 236.48709, -8.170532, 0.07139679]
[6450, 238.09537, 228.87477, -9.220597, 0.0695029]
[6460, 235.81297, 225.99872, -9.814248, 0.06934278]
[6470, 247.44928, 238.73935, -8.709931, 0.0745626]
[6480, 256.9659, 247.1991, -9.766811, 0.07800647]
[6490, 244.76324, 235.5535, -9.209753, 0.07247247]
[6500, 263.0251, 254.57468, -8.450424, 0.080639355]
[6510, 235.5001, 2

In [ ]:
real_images_split = batch_split(np_x, n_squares_side, n_channels)
stack_real_images_split = stack_batch_split(real_images_split)
print(stack_real_images_split.shape)

In [ ]:
batch_size_display = 10
hard = np.zeros(sp2.shape)
ind=1
scrambled_pieces = unflatten_batch(scrambled_pieces_split_np, n_channels)
joined_scrambled = join_batch_split(scrambled_pieces)
for i in range(batch_size_display):
    hard[i,0,:,:] =soft_to_hard(np.transpose(sgen[ind][i,0,:,:]))
    
unscrambled_images_hard =  np.matmul(hard[:batch_size_display,0,:,:], np.reshape(scrambled_pieces[:batch_size_display,:,:,:], [batch_size_display,n_squares_side **2 ,-1]))
unscrambled_images_soft =  ogen[ind]
unflatten_inf_hard = unflatten_batch(unscrambled_images_hard, n_channels)
joined_inf_hard = join_batch_split(unflatten_inf_hard)
unflatten_inf_soft = unflatten_batch(unscrambled_images_soft, n_channels)
joined_inf_soft = join_batch_split(unflatten_inf_soft)


In [ ]:
fig,ax=plt.subplots(10,3,figsize=(10,30))
#plt.imshow(joined_inf[0,:,:,0])
print(np_x.shape)
for i in range(10):
    
    
    ax[i,1].imshow(joined_inf_hard[i,:,:,0],'gray')
    ax[i,2].imshow(joined_inf_soft[i,:,:,0],'gray')
    ax[i,0].imshow(joined_scrambled[i,:,:,0],'gray')
    for k in range(3):
        ax[i,k].get_xaxis().set_visible(False)
        ax[i,k].get_yaxis().set_visible(False)

In [ ]:
print(np.sum(ssp2[0,0,:,:], axis=1))
plt.figure(figsize=(20,20))
plt.imshow(ssp2[5,0,:,:])
print(ssp2[0,0,0,:])

In [ ]:
print(np_x)

In [ ]:

joined_inf_hard = join_batch_split(unflatten_batch(o2, n_channels))